
# Alpha–Beta Pruning on Mancala (State-Based)

We’ll upgrade Minimax with **Alpha–Beta pruning** and show how **move ordering** reduces the number of expanded nodes. We continue to use the state-based engine:
- `legal_actions(state)`
- `step(state, action)`
- `evaluate(state)`



## 0) Setup


In [1]:

import os, sys, pathlib
REPO_ROOT = pathlib.Path().resolve().parents[1]  # .../tutorials/minimax_alpha/ -> repo root
SRC = REPO_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from mancala_ai.engine.core import new_game, legal_actions, step, evaluate
print("Imports OK. Repo root:", REPO_ROOT)


Imports OK. Repo root: /mnt/c/Users/ndqua/OneDrive - Trent University/AMOD 5310H-AI/AI-algorithm-for-Mancala-game



## 1) Helpers


In [2]:

import copy, math
from typing import Dict, Optional, Tuple, List

def score_for(state: Dict, root_idx: int) -> float:
    s = copy.deepcopy(state)
    s["current_player"] = root_idx
    return float(evaluate(s))

class Stats:
    def __init__(self):
        self.visits = 0

def print_state(s):
    p0, p1 = s["pits"][0], s["pits"][1]
    st0, st1 = s["stores"]
    turn = s["current_player"]
    print("+----- Mancala -----+")
    print("P1 store:", st1)
    print("P1 pits: ", list(reversed(p1)))
    print("P0 pits: ", p0)
    print("P0 store:", st0)
    print("Turn: P", turn, sep="")
    print("+-------------------+")



## 2) Simple one-ply ordering (optional but recommended)

We order actions by the one-ply score of the resulting state (from the root player’s perspective).


In [3]:

def order_moves_simple(state: Dict, root_idx: int) -> List[int]:
    acts = legal_actions(state)
    scored = []
    for a in acts:
        ns, _, _ = step(state, a)
        scored.append((score_for(ns, root_idx), a))
    scored.sort(reverse=True)
    return [a for _, a in scored]



## 3) Alpha–Beta with correct extra-turn handling


In [ ]:

def alphabeta(state: Dict, depth: int, alpha: float, beta: float, root_idx: int, stats: Stats,
              use_ordering: bool = True):
    stats.visits += 1

    if depth == 0 or sum(state["pits"][0]) == 0 or sum(state["pits"][1]) == 0:
        return score_for(state, root_idx), None

    acts = legal_actions(state)
    if not acts:
        return score_for(state, root_idx), None

    is_max = (state["current_player"] == root_idx)
    ordered = order_moves_simple(state, root_idx) if use_ordering else acts
    best_move = ordered[0]

    if is_max:
        val = -math.inf
        a = alpha
        for mv in ordered:
            ns, _, _ = step(state, mv)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            child_v, _ = alphabeta(ns, depth - reduce, a, beta, root_idx, stats, use_ordering)
            if child_v > val:
                val, best_move = child_v, mv
            a = max(a, val)
            if beta <= a:  # beta cut
                break
        return val, best_move
    else:
        val = math.inf
        b = beta
        for mv in ordered:
            ns, _, _ = step(state, mv)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            child_v, _ = alphabeta(ns, depth - reduce, alpha, b, root_idx, stats, use_ordering)
            if child_v < val:
                val, best_move = child_v, mv
            b = min(b, val)
            if b <= alpha:  # alpha cut
                break
        return val, best_move

def choose_move_alphabeta(state: Dict, depth: int = 7, use_ordering: bool = True):
    """Convenience wrapper: search and return a move for the current player.

    Parameters
    ----------
    state : dict
        Current Mancala state (see `minimax` docstring).
    depth : int, default=5
        Search depth in plies.

    Returns
    -------
    (move, stats) : (int, Stats)
        - move: selected pit index (0..5). If no legal moves exist, returns 0.
        - stats: node expansion statistics for this search.

    Notes
    -----
    • If something goes wrong and `minimax` returns None for the move (e.g.,
      at a leaf), we fall back to the first legal action to keep the game flowing.
    """
    
    stats = Stats()
    _, mv = alphabeta(state, depth, -1e9, 1e9, state["current_player"], stats, use_ordering=use_ordering)
    if mv is None:
        acts = legal_actions(state)
        mv = int(acts[0]) if acts else 0
    return int(mv), stats



## 4) Experiments

Compare node counts for Minimax vs Alpha–Beta, and Alpha–Beta with vs without ordering.


In [9]:
import random

# Baseline state
s = new_game()

ns = s
for _ in range(4):
    if s["current_player"] == 0:
        mv_ab, _ = choose_move_alphabeta(ns, depth=5, use_ordering=True)
        ns, _, _ = step(ns, mv_ab)
    else:
        mv_random = random.choice(legal_actions(ns))
        ns, _, _ = step(ns, mv_random)
    print_state(ns)

# Plain minimax for comparison (implemented inline for this cell)
class _MStats: 
    def __init__(self): self.visits = 0

def _minimax(state: Dict, depth: int, root_idx: int, st: _MStats):
    st.visits += 1
    if depth == 0 or sum(state["pits"][0]) == 0 or sum(state["pits"][1]) == 0:
        return score_for(state, root_idx), None
    acts = legal_actions(state)
    if not acts: return score_for(state, root_idx), None
    is_max = (state["current_player"] == root_idx)
    best_mv = acts[0]
    if is_max:
        val = -math.inf
        for mv in acts:
            ns, _, _ = step(state, mv)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            v, _ = _minimax(ns, depth - reduce, root_idx, st)
            if v > val: val, best_mv = v, mv
        return val, best_mv
    else:
        val = math.inf
        for mv in acts:
            ns, _, _ = step(state, mv)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            v, _ = _minimax(ns, depth - reduce, root_idx, st)
            if v < val: val, best_mv = v, mv
        return val, best_mv

st_min = _MStats()
_, mv_min = _minimax(ns, depth=5, root_idx=ns["current_player"], st=st_min)
print("Minimax   -> nodes:", st_min.visits, "move:", mv_min)

mv_ab_no, st_ab_no = choose_move_alphabeta(ns, depth=5, use_ordering=False)
print("AlphaBeta (no order) -> nodes:", st_ab_no.visits, "move:", mv_ab_no)

mv_ab, st_ab = choose_move_alphabeta(ns, depth=5, use_ordering=True)
print("AlphaBeta (ordered)  -> nodes:", st_ab.visits, "move:", mv_ab)


+----- Mancala -----+
P1 store: 0
P1 pits:  [4, 4, 4, 4, 5, 5]
P0 pits:  [4, 4, 4, 4, 0, 5]
P0 store: 1
Turn: P1
+-------------------+
+----- Mancala -----+
P1 store: 1
P1 pits:  [5, 5, 0, 4, 5, 5]
P0 pits:  [5, 4, 4, 4, 0, 5]
P0 store: 1
Turn: P0
+-------------------+
+----- Mancala -----+
P1 store: 1
P1 pits:  [5, 5, 1, 5, 6, 6]
P0 pits:  [5, 4, 4, 4, 0, 0]
P0 store: 2
Turn: P1
+-------------------+
+----- Mancala -----+
P1 store: 2
P1 pits:  [6, 6, 2, 6, 7, 0]
P0 pits:  [5, 4, 4, 4, 0, 0]
P0 store: 2
Turn: P1
+-------------------+
Minimax   -> nodes: 189339 move: 4
AlphaBeta (no order) -> nodes: 25960 move: 4
AlphaBeta (ordered)  -> nodes: 13151 move: 4



## 5) Exercise — Instrument and compare

1. Change `depth` (e.g., 3, 5, 7) and record node counts.  
2. Add a **very simple move ordering**: try actions that **reach the store** first (i.e., `pits[i] == 6-i`).  
   Measure if node counts change.


In [ ]:

# TODO: implement a simple move ordering inside minimax.
# HINT:
# actions = sorted(actions, key=lambda i: state["pits"][state["current_player"]][i] == (6 - i), reverse=True)
# Then rerun choose_move_minimax and compare Stats().visits.

# --- helper: order actions (prefer extra-turn moves, then more stones) ---
def _order_actions(state: Dict, actions):
    row = state["current_player"]
    pits = state["pits"][row]
    # Key returns a tuple:
    #   (lands_in_store?, stones_in_pit)
    # We sort descending so True > False and larger stone counts first.
    def key(i: int):
        lands_store = (pits[i] == (6 - i))
        return (lands_store, pits[i])
    return sorted(actions, key=key, reverse=True)

def minimax(state: Dict, depth: int, root_idx: int, stats: Stats) -> Tuple[float, Optional[int]]:
    stats.visits += 1

    # Leaf or terminal
    if depth == 0 or sum(state["pits"][0]) == 0 or sum(state["pits"][1]) == 0:
        return score_for(state, root_idx), None

    actions = legal_actions(state)
    if not actions:
        return score_for(state, root_idx), None

    # >>> NEW: apply ordering <<<
    actions = _order_actions(state, actions)

    is_max = (state["current_player"] == root_idx)
    best_move = actions[0]

    if is_max:
        best = -math.inf
        for a in actions:
            ns, _, _ = step(state, a)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            v, _ = minimax(ns, depth - reduce, root_idx, stats)
            if v > best:
                best, best_move = v, a
        return best, best_move
    else:
        best = math.inf
        for a in actions:
            ns, _, _ = step(state, a)
            reduce = 0 if ns["current_player"] == state["current_player"] else 1
            v, _ = minimax(ns, depth - reduce, root_idx, stats)
            if v < best:
                best, best_move = v, a
        return best, best_move
pass


In [ ]:
mv, st = alphabeta(new_game(), depth=5)
print("nodes (no ordering):", st.visits)


## 6) (Optional) Iterative Deepening + Time Budget

Implement iterative deepening (depth = 1..D) with a time cutoff; return the best move from the deepest completed search.


In [ ]:

# TODO: iterative deepening scaffold
# import time
# def choose_move_id(state, max_depth=10, time_limit=1.0):
#     start = time.time()
#     best_mv = 0
#     for d in range(1, max_depth+1):
#         if time.time() - start > time_limit: break
#         mv, st = choose_move_alphabeta(state, depth=d, use_ordering=True)
#         best_mv = mv
#     return best_mv
pass
